<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/notebooks/binary_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Neural Networks for Fundamental Binary Functions

This is nice illustration that a neural network can compute anything.


In [0]:
class Context:
    VERBOSE=False
    QUIET=False

In [0]:
#@title Import Directives
from contextlib import redirect_stderr
import os

import tensorflow as tf

# Suppress Tensorflow log spew.
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Suppress Keras log spew.
with redirect_stderr(open(os.devnull, "w")):
    import keras

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

import numpy as np
import argparse

In [0]:
#@title Text Formatting and Output Functions
def fmt(f, *args):
    if len(args) == 0:
        return str(f)
    else:
        if type(f) is str:
            return f.format(*args)
        else:
            return [str(f)] + [str(s) for s in args]

def log(f, *args):
    print(fmt(f, *args))

def out(f, *args):
    if not Context.QUIET:
        log(fmt(f, *args))

In [0]:
Binary_functions = {
    'not' : { 'function' : 'Binary NOT', 'X' : np.array([[0],[1]]),                 'Y' : np.array([[1],[0]]) },
    'xor' : { 'function' : 'Binary XOR', 'X' : np.array([[0,0],[0,1],[1,0],[1,1]]), 'Y' : np.array([[0],[1],[1],[0]]) },
    'and' : { 'function' : 'Binary AND', 'X' : np.array([[0,0],[0,1],[1,0],[1,1]]), 'Y' : np.array([[0],[0],[0],[1]]) },
    'or'  : { 'function' : 'Binary OR',  'X' : np.array([[0,0],[0,1],[1,0],[1,1]]), 'Y' : np.array([[0],[1],[1],[1]]) }
}

In [5]:
Binary_functions.keys()

dict_keys(['not', 'xor', 'and', 'or'])

In [0]:
def compute(function, X, Y):
    if Context.VERBOSE:
        out("X.shape={}", X.shape)
        out("Input Array of {} Inputs:\n{}", function, X)
        out("Ouput Array of Truth Results:\n{}".format(Y))

    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1))

    out("--------------------------------------------------------------------------")
    out("----- Finding {} Mapping Function using a Brute Force Neural Network:", function)

    model.fit(X, Y, batch_size=1, epochs=1000, verbose=1 if Context.VERBOSE else 0)

    out("----- Keras information on the function:")
    if not Context.QUIET: model.summary()

    log("--------------------------------------------------------------------------")
    log("Predicted {} Truth Values from input table:\n{}\nActual results:\n{}", function, model.predict(X), Y)

In [0]:
def main():
    for f in Binary_functions.values():
        compute(**f)

In [0]:
#@title Python Command Line Main Program 
try:
    get_ipython
except NameError:
    if __name__=='__main__':
        desc = "This is a brute force method that demonstrates a concept rather than a practical example."
        parser = argparse.ArgumentParser(
            description = "Keras based Neural Network to train for and evaluate Boolean Functions.",
            epilog = desc,
        )    
        parser.add_argument('-v', "--verbose", help="increase output verbosity.", action='store_true')
        parser.add_argument('-q', "--quiet",   help="decrease output verbosity.", action='store_true')
        parser.add_argument('-?',              help=argparse.SUPPRESS,            action='store_true')

        ap = parser.parse_args()
        if ap.verbose: Context.VERBOSE=True
        if ap.quiet:   Context.QUIET=True
        if getattr(ap, '?', False):
            parser.print_help()
            exit()
        main()

In [13]:
Context.VERBOSE = False
Context.QUIET = False
compute(**Binary_functions['xor'])

--------------------------------------------------------------------------
----- Finding Binary XOR Mapping Function using a Brute Force Neural Network:
----- Keras information on the function:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 8)                 24        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 9         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
--------------------------------------------------------------------------
Predicted Binary XOR Truth Values from input table:
[[0.00232605]
 [0.99550736]
 [0.99591994]
 [0.00522789]]
Actual results:
[[0]
 [1]
 [1]
 [0]]
